In [1]:
from bs4 import BeautifulSoup
import getNDVI
import getSAR
import common
import numpy as np
import ee
import getPair
from datetime import datetime
import disp_multiple_images

%matplotlib inline

In [2]:
# Trigger the authentication flow.
#ee.Authenticate()

In [3]:
# Initialize the library.
ee.Initialize()

In [4]:
def parseKML(inputfile):
  with open(inputfile, "r") as f:
    soup = BeautifulSoup(f, features="html.parser")
    return [np.array([i.split(",")[0:2] for i in node.string.split()]).astype(np.float).tolist()\
            for node in soup.findAll("coordinates")]

In [6]:
if __name__ == "__main__":
  for array in parseKML("2017_polygons.kml"):
    startDate, EndDate = "2017-01-01","2017-02-01"    
    data, _ = getPair.arrayToPairs(array, startDate, EndDate, False, 0.5)
    
    px_pairs = None
    for i in range(0, len(data), 3):
        # NDVI, SAR
        tmp = np.dstack((data[i][0], data[i+1][0], data[i+2][0]))
        tmp = tmp.reshape(-1, tmp.shape[-1])
        tmp = tmp[np.all(tmp != 0, axis=1)]
        px_pairs = np.concatenate((px_pairs, tmp), axis = 0) if not px_pairs is None else tmp
    
    print(px_pairs)
    # px_pairs is now pixel-pairs for a given field over a given time span
    
    break

{0: [0], 1: [0], 2: [0], 3: [0], 4: [1], 5: [1]}
[[ 6.16822429e-02 -1.91848335e+01 -8.02712822e+00]
 [ 2.78884470e-02 -1.94849224e+01 -6.78397465e+00]
 [ 9.11854114e-03 -1.81715927e+01 -4.95610952e+00]
 ...
 [ 8.52546915e-02 -2.19176769e+01 -8.65106869e+00]
 [ 3.11158802e-02 -2.04505444e+01 -1.07489624e+01]
 [ 4.42967899e-02 -1.97389812e+01 -1.22920523e+01]]
